In [2]:
import sqlite3

In [3]:
dbname = 'input_data/2021.db'
conn = sqlite3.connect(dbname)
cur = conn.cursor()

In [7]:
cur.execute("SELECT name FROM sqlite_master WHERE TYPE='table'")
cur.fetchall()

[('pitch_data',),
 ('data_at_bat',),
 ('player',),
 ('game_data',),
 ('batting_stats',),
 ('pitch_stats',)]

In [9]:
cur.execute('select result_big from data_at_bat')
result_big_list = [result[0] for result in cur.fetchall()]
# ngワードが入っているリザルトの除去(本塁打が別れてるのはランニング本塁打があるから)
ng_words = ['けん制', '【代打】', '【代走】', '【守備】', '【継投】', 'ボーク', 'スリーバント', '見逃し', '空振り', '敬遠', '振り逃げ',
            'ファウル', 'ボール', '三振', '四球', '死球', '守備妨害', '打撃妨害', '左本塁打', '中本塁打', '右本塁打']
result_big_list = [result for result in result_big_list if all([ng_word not in result for ng_word in ng_words])]
# 打点の除去
result_big_list = [result.replace('＋1点', '').replace('＋2点', '').replace('＋3点', '').replace('＋4点', '').rstrip() for result in result_big_list]
print(sorted(list(set(result_big_list))))

['', '一2塁打', '一エラー', '一ゴロ', '一フライ', '一ライナー', '一併殺打', '一安打', '一犠打', '一犠打野選', '一野選', '三2塁打', '三エラー', '三ゴロ', '三フライ', '三ライナー', '三併殺打', '三安打', '三犠打', '三犠打野選', '三野選', '中2塁打', '中3塁打', '中エラー', '中ゴロ', '中フライ', '中ランニング本塁打', '中安打', '中犠飛', '二エラー', '二ゴロ', '二フライ', '二ライナー', '二併殺打', '二安打', '二犠打', '二野選', '右2塁打', '右3塁打', '右エラー', '右ゴロ', '右フライ', '右安打', '右犠飛', '左2塁打', '左3塁打', '左エラー', '左ゴロ', '左フライ', '左安打', '左犠飛', '投エラー', '投ゴロ', '投フライ', '投ライナー', '投併殺打', '投安打', '投犠打', '投犠打野選', '投野選', '捕エラー', '捕ゴロ', '捕フライ', '捕併殺打', '捕安打', '捕犠打', '捕犠打野選', '犠打失', '犠飛失', '空振り', '遊2塁打', '遊エラー', '遊ゴロ', '遊フライ', '遊ライナー', '遊併殺打', '遊安打', '遊野選']


In [44]:
cur.execute('select dab.result_big, pd.pitcher_id, pd.c, pd.first, pd.second, pd.third, pd.ss, pd.lf, pd.cf, pd.rf, dab.out from pitch_data as pd join data_at_bat as dab on pd.id_at_bat = dab.id group by id_at_bat having number_pitch_at_bat = max(number_pitch_at_bat)')
data = cur.fetchall()

In [45]:
import pandas as pd

df = pd.DataFrame(data, columns=['result', 'pitcher', 'c', 'first', 'second', 'third', 'ss', 'lf', 'cf', 'rf', 'out'])

In [46]:
df

,result,pitcher,c,first,second,third,ss,lf,cf,rf,out
0,投ゴロ,1200041,1700044,1500015,1700047,1400101,700003,1600070,800043,700027,1
1,三ゴロ,1200041,1700044,1500015,1700047,1400101,700003,1600070,800043,700027,1
2,四球,1200041,1700044,1500015,1700047,1400101,700003,1600070,800043,700027,0
3,四球,1600081,1300093,2000035,1700046,1200053,1500063,1600089,1100061,1300038,0
4,左安打 ＋1点,1600081,1300093,2000035,1700046,1200053,1500063,1600089,1100061,1300038,0
...,...,...,...,...,...,...,...,...,...,...,...
64131,右本塁打 ＋1点,1600012,600026,600020,900056,1100103,1700034,1600094,1800117,1100130,0
64132,空振り三振,1600012,600026,600020,900056,1100103,1700034,1600094,1800117,1100130,1
64133,四球,700065,600026,600020,900056,1100103,1700034,1800122,1100130,1800123,0
64134,左安打,700065,600026,600020,900056,1100103,1700034,1800122,1100130,1800123,0


In [47]:
rbi_strip = lambda x: x.replace('＋1点', '').replace('＋2点', '').replace('＋3点', '').replace('＋4点', '').rstrip()
df['result'] = df['result'].map(rbi_strip)

In [48]:
infield_df = df[df['result'].isin(['一2塁打', '一エラー', '一ゴロ', '一フライ', '一ライナー', '一併殺打', '一安打', '一犠打', '一犠打野選', '一野選', '遊2塁打', '遊エラー', '遊ゴロ', '遊フライ', '遊ライナー', '遊併殺打', '遊安打', '遊野選', '中2塁打', '中3塁打', '中エラー', '中ゴロ', '中フライ', '中ランニング本塁打', '中安打', '中犠飛', '二エラー', '二ゴロ', '二フライ', '二ライナー', '二併殺打', '二安打', '二犠打', '二野選', '右2塁打', '右3塁打', '右エラー', '右ゴロ', '右フライ', '右安打', '右犠飛', '左2塁打', '左3塁打', '左エラー', '左ゴロ', '左フライ', '左安打', '左犠飛', '投エラー', '投ゴロ', '投フライ', '投ライナー', '投併殺打', '投安打', '投犠打', '投犠打野選', '投野選', '捕エラー', '捕ゴロ', '捕フライ', '捕併殺打', '捕安打', '捕犠打', '捕犠打野選', '犠打失', '犠飛失', '遊2塁打', '遊エラー', '遊ゴロ', '遊フライ', '遊ライナー', '遊併殺打', '遊安打', '遊野選', '二エラー', '二ゴロ', '二フライ', '二ライナー', '二併殺打', '二安打', '二犠打', '二野選', '三2塁打', '三エラー', '三ゴロ', '三フライ', '三ライナー', '三併殺打', '三安打', '三犠打', '三犠打野選', '三野選'])]

In [49]:
infield_df

,result,pitcher,c,first,second,third,ss,lf,cf,rf,out
0,投ゴロ,1200041,1700044,1500015,1700047,1400101,700003,1600070,800043,700027,1
1,三ゴロ,1200041,1700044,1500015,1700047,1400101,700003,1600070,800043,700027,1
4,左安打,1600081,1300093,2000035,1700046,1200053,1500063,1600089,1100061,1300038,0
5,中2塁打,1600081,1300093,2000035,1700046,1200053,1500063,1600089,1100061,1300038,0
7,右安打,1600081,1300093,2000035,1700046,1200053,1500063,1600089,1100061,1300038,0
...,...,...,...,...,...,...,...,...,...,...,...
64126,右安打,1600164,1000176,800081,1600167,1700136,1000175,1400127,1900104,1000128,0
64127,遊ゴロ,1600164,1000176,800081,1600167,1700136,1000175,1400127,1900104,1000128,1
64128,右安打,1600164,1000176,800081,1600167,1700136,1000175,1400127,1900104,1000128,0
64130,遊フライ,1600012,600026,600020,900056,1100103,1700034,1600094,1800117,1100130,1


In [ ]:
tmp_dict = {}
for index, row in infield_df.iterrows():
    tmp_list = []

    # 責任が投手のみ
    if row['result'] in ['hoge']:
        tmp_dict.setdefault([row['pitcher'],], [])
        tmp_list.append(row['out'])